In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, concatenate, Flatten, Dropout
from tensorflow.keras.models import Model

In [ ]:
def run_parallel(data_shape, batched_train_data, batched_val_data, train_slices, 
             val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET):
    model = compile_model(data_shape, FUTURE_TARGET)
    training_history, training_time = fit_model(model, batched_train_data, batched_val_data, train_slices, 
                                                val_slices, BATCH_SIZE, EPOCHS)
    return model, training_history, training_time

In [ ]:
def compile_model(data_shape, FUTURE_TARGET):
    
    inputs = Input(shape=data_shape, name='main_input')
    conv = Conv1D(filters=50,kernel_size=10,strides=2,padding='same',activation='relu')(inputs)
    lstm = LSTM(32,return_sequences=False)(inputs) #true if deep lstm
    x = Flatten()(conv)
    concat = concatenate([lstm,x])
    main_output = Dense(1,name='main_output')(concat)
    model = Model(inputs=[inputs], outputs=[main_output])
    
    model.summary()
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss={'main_output':'mae'},
                  loss_weights={'main_output':1}, metrics=['mae', 'mape', 'mse'])
    return model

In [ ]:
def fit_model(model, batched_train_data, batched_val_data, train_slices, val_slices, BATCH_SIZE, EPOCHS):
    train_batches = train_slices // BATCH_SIZE 
    val_batches = val_slices // BATCH_SIZE
    # Splits the dataset into batches of this size: we perform gradiant descent once per batch
    start = timer()
    training_history = model.fit(batched_train_data, epochs=EPOCHS, 
                                 steps_per_epoch=train_batches,
                                 validation_data=batched_val_data,
                                 validation_steps=val_batches)
    end = timer()
    return training_history, end - start # time in seconds
